# Automação e Web Scraping com Selenium

#### <strong>Introdução</strong>
Este projeto foi realizado por Gabriel Will, a fim de estudos.<br>
Site: <strong><a href="https://gabrielwill.com.br">gabrielwill.com.br</a></strong><br>
Linkedin: <strong><a href="https://www.linkedin.com/in/gabrielwillgg/">Gabriel Will</a></strong>

#### <strong>Objetivo</strong>
O objetivo deste notebook é desenvolver um web scraping, utilizando Selenium, que pesquisa produtos no Google Shopping a partir de uma planilha de Excel, filtrando através de preço mínimo e preço máximo e também termos para excluir alguns resultados indesejados.

Importamos dos pacotes que usaremos para desenvolver este projeto, sendo elas:
- Selenium: o pacote principal do projeto, usado para extrair os dados da página.
- Pandas: poderosíssima biblioteca para podermos gerar um DataFrame e visualizar o resultado de nosso web scraping.

In [1]:
# Importando pacotes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
# Importando e transformando os produtos em DataFrame
produtos_df = pd.read_excel("buscas.xlsx")
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 15 128 gb,mini watch usado,4000,5000
1,rtx 4060,3060,1300,2800
2,ryzen 5 5600g,NaN,400,1000
3,placa mãe b550m,NaN,300,1000
4,NaN,NaN,300,200


In [3]:
#Tratando o DataFrame pra substituir as células vazias por outro valor
produtos_df = produtos_df.fillna('-9999')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 15 128 gb,mini watch usado,4000,5000
1,rtx 4060,3060,1300,2800
2,ryzen 5 5600g,-9999,400,1000
3,placa mãe b550m,-9999,300,1000
4,-9999,-9999,300,200


In [4]:
# Cria o navegador automatizado
nav = webdriver.Chrome()

In [5]:
# Cria função para automatizar a busca e extração de dados
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = str(produto)
    termos_banidos = str(termos_banidos)
    # filtra o nome do produto, caso tenha um espaço ele transforma em uma lista com todas as palavras do produto
    lista_termos_nome = []
    if " " in produto:
        lista_termos_nome = produto.split(" ")
    else:
        lista_termos_nome.append(produto)
    # filtra os termos do produto
    lista_termos_banidos = []
    if " " in termos_banidos:
        lista_termos_banidos = termos_banidos.split(" ")
    elif termos_banidos != '-9999':
        lista_termos_banidos.append(termos_banidos)
    
    # Entra no site do google shopping e digita o nome do produto
    nav.get('https://shopping.google.com.br/')
    nav.find_element(By.ID, 'REsRA').send_keys(produto, Keys.ENTER)
    lista_resultados = nav.find_elements(By.CLASS_NAME,'sh-dgr__content')

    lista_ofertas = []
    # Extrai todas as informações dos produtos e adiciona em uma lista de tuplas
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME,'tAxDx').text

        tem_termos_banidos = False
        if len(lista_termos_banidos) >= 1:
            for palavra in lista_termos_banidos:
                palavra = str(palavra)
                if palavra.lower() in nome.lower():
                    tem_termos_banidos = True

        tem_termos_nome_produtos = True
        for palavra in lista_termos_nome:
            palavra = str(palavra)
            if palavra.lower() not in nome.lower():
                tem_termos_nome_produtos = False

        if tem_termos_banidos or not tem_termos_nome_produtos:
            continue
        
        # tratando os preços
        preco = resultado.find_element(By.CLASS_NAME,'a8Pemb').text
        preco = preco.replace("R$","").replace(" ","").replace(".","")
        preco = float(preco.replace(",","."))
        if preco_minimo <= preco <= preco_maximo:
            link = resultado.find_element(By.CSS_SELECTOR, "a[class='shntl']").get_attribute('href')
            lista_ofertas.append((nome, preco, link))
    return(lista_ofertas)


In [6]:
lista_google_shopping = {}
# loop para acessar os indexes do DataFrame principal
for row in produtos_df.index:
    produto = produtos_df.loc[row, "Nome"]
    if produto == '-9999':
        continue
    termos_banidos = produtos_df.loc[row, "Termos banidos"]

    preco_minimo = float(produtos_df.loc[row, "Preço mínimo"])
    preco_maximo = float(produtos_df.loc[row, "Preço máximo"])

    pesquisa_ofertas = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    lista_google_shopping[produto] = pesquisa_ofertas

Exportando para o Excel.

In [7]:
writer = pd.ExcelWriter('ofertas_google_shopping.xlsx')
# loop para poder criar páginas da planilha com os nomes dos produtos
for produto in lista_google_shopping:
    df = pd.DataFrame(lista_google_shopping[produto], columns=['produto', 'preco', 'link'])
    df.to_excel(writer, sheet_name=produto, index=False)
writer.close()

Depois de todo esse processo, podemos também analisar os preços de determinado produto, sabendo qual é o seu preço mínimo, máximo e a média de preços.

In [8]:
produto = "rtx 4060"
item_df = pd.DataFrame(lista_google_shopping[produto], columns=['produto', 'preco', 'link']).set_index("produto")
item_df.head()

,preco,link
produto,,
Placa De Vídeo NVIDIA Gigabyte GeForce RTX 4060 Windforce OC 8GB GDDR6,1899.99,https://www.google.com.br/url?url=https://www....
Placa De Vídeo Galax GeForce RTX 4060 8GB 1-Click OC 1x GDDR6 - 46NSL8MD8ZOC,1689.99,https://www.google.com.br/url?url=https://www....
Placa De Vídeo Gigabyte GeForce RTX 4060 Eagle OC 8GB GDDR6,1999.90,https://www.google.com.br/url?url=https://www....
Placa De Video Gigabyte GeForce RTX 4060 Gaming OC 8GB GDDR6,2579.91,https://www.google.com.br/url?url=https://www....
Placa De Vídeo Gigabyte GeForce RTX 4060 8GB GDDR6 Aero OC,2099.99,https://www.google.com.br/url?url=https://www....


In [9]:
menor_preco = item_df[item_df['preco'] == item_df['preco'].min()]
maior_preco = item_df[item_df['preco'] == item_df['preco'].max()]

print(f'O item de menor preço: {menor_preco.index[0]} custando {menor_preco.iloc[0].iloc[0]}.\nLink: {menor_preco.iloc[0].iloc[1]}.\n')
print(f'O item de maior preço: {maior_preco.index[0]} custando {maior_preco.iloc[0].iloc[0]}.\nLink: {maior_preco.iloc[0].iloc[1]}.\n')
print(f'A média dos preços: {(item_df['preco'].mean()):.2f}')

O item de menor preço: Placa De Vídeo Galax GeForce RTX 4060 8GB 1-Click OC 1x GDDR6 - 46NSL8MD8ZOC custando 1689.99.
Link: https://www.google.com.br/url?url=https://www.kabum.com.br/produto/496012/placa-de-video-rtx-4060-1-click-oc-1x-galax-nvidia-geforce-8gb-gddr6-dlss-ray-trancing-g-sync%3Fsrsltid%3DAfmBOop-h6ZyWFlL22xhPusfMVeJZ0h1LH4qArFnJUGrWDdn1GgqDcvjgXA&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwiBucSYsuWFAxXSrJUCHQFLDVQQguUECMkN&usg=AOvVaw1tmNh5ddkXuQ5a3N3FPT4n.

O item de maior preço: Placa De Vídeo msi GeForce RTX 4060 Ti Gaming X, 8GB, GDDR6 - 912-V515-022 custando 2794.9.
Link: https://www.google.com.br/url?url=https://www.pontofrio.com.br/placa-de-video-rtx-4060-ti-gaming-x-8g-msi-nvidia-geforce-8-1562804540/p/1562804540%3Futm_medium%3Dcpc%26utm_source%3Dgoogle_freelisting%26IdSku%3D1562804540%26idLojista%3D11578%26tipoLojista%3D3P&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwiBucSYsuWFAxXSrJUCHQFLDVQQguUECPgQ&usg=AOvVaw2AbJETgT6QMXQvDob09tg7.

A média dos preços: 2213.